In [1]:
# Quick hack to load local SDK code
import os
os.getcwd()
os.chdir("/Users/andres/code/validmind-sdk")

In [2]:
# Load API key and secret from environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import validmind as vm

In [5]:
vm.init(project="cl1jyv16o000809lg98gi9tie")

{'stauts': 'OK'}